# Tutorial: Nash-Sutcliffe Efficiency (NSE) Calculation
In this tutorial, we will explore the Nash-Sutcliffe Efficiency (NSE) calculation function `scores.continuous.nse`.

NSE is a widely used metric in hydrology and other fields to evaluate the performance of a model by comparing its predictions to observed data.

#### NSE

The Nash-Sutcliffe Efficiency (NSE) is defined as:

$\text{NSE} = 1 - \huge\frac{\huge\sum_{i=1}^{n}w_i.(f_i - o_i)^2}{\huge\sum_{i=1}^{n}w_i.(o_i - \bar{o})^2}$

Where:
- $f_i$ is the forecast or predicted value
- $o_i$ is the observed value
- $\bar{o}$ is the mean of the observed values
- $n$ is the number of data points
- $w$ is an optional weighting that can be associated to the error terms, where $\forall i \ : \ 0 \leq w_i\ , \ \text{and there exists at least one}\ \  i \ : \ 0 \lt w_i$

> _**Note**_
> 
> while $i$ above is shown as a integer index from 0 to n, its just as valid to use a multi index e.g. $i: (0, 0) \rightarrow (M, N)$. If one wishes to accumulate the NSE scores over multiple indices, the `reduce_dims` and `preserve_dims` can be used (similar to other scores)

A perfect model has an NSE value of 1, while a model performing as poorly as the mean of the observed data has an NSE value of 0 or less.

In hydrological modeling, the Nash–Sutcliffe Efficiency (NSE) is essential for assessing model performance. An NSE of 1 indicates perfect prediction, while 0 suggests the model performs as well as predicting the mean of the data. Negative values imply the observed mean is a better predictor. NSE values closer to 1 denote superior predictive ability. In regression analyses, NSE parallels the coefficient of determination (R²), representing model fit on a scale from 0 to 1.

**References:**
1. Nash, J. E., & Sutcliffe, J. V. (1970). River flow forecasting through conceptual models part I — A discussion of principles.<br>
   Journal of Hydrology (Vol. 10, Issue 3, pp. 282–290). Elsevier BV.<br>
   _doi:10.1016/0022-1694(70)90255-6_
2. Hundecha, Y., & Bárdossy, A. (2004). Modeling of the effect of land use changes on the runoff generation of a river basin through parameter regionalization of a watershed model.<br>
   Journal of Hydrology, 292(1-4), 281-295.<br>
   _doi:10.1016/j.jhydrol.2004.01.002_


## Using the `nse` Function

Let's start by importing the `nse` function from our module and exploring its usage with different types of input data.

In [39]:
from scores.continuous import nse
import numpy as np
import xarray as xr
import pandas as pd

np.random.seed(0)  # set the seed to make notebook reproducible

**Example 1**: Xarray DataArray

In [2]:
fcst_xr = xr.DataArray([3, 4, 5, 6, 7])
obs_xr = xr.DataArray([2, 3, 4, 5, 6])
nse_xr = nse(fcst_xr, obs_xr)
print("NSE for Xarray DataArray:", nse_xr)

NSE for Xarray DataArray: <xarray.DataArray ()> Size: 8B
array(0.5)


**Example 2**: Large Xarray DataArrays

In [3]:

fcst_large = xr.DataArray(
    data=np.random.random_sample((1000, 1000)) * 360,
    dims=["space", "time"],
    coords=[np.arange(0, 1000), np.arange(0, 1000)],
)
obs_large = xr.DataArray(
    data=np.random.random_sample((1000, 1000)) * 360,
    dims=["space", "time"],
    coords=[np.arange(0, 1000), np.arange(0, 1000)],
)
nse_large = nse(fcst_large, obs_large)
print("NSE for large Xarray DataArrays:", nse_large)

NSE for large Xarray DataArrays: <xarray.DataArray ()> Size: 8B
array(-0.9995806)


**Example 3**: Angular and array

In [5]:
fcst_xr = xr.DataArray([3, 4, 5, 6, 7])
obs_xr = xr.DataArray([2, 3, 4, 5, 6])
nse_anular = nse(fcst_xr, obs_xr, is_angular=True)
print("NSE for angular types:", nse_anular)

NSE for angular types: <xarray.DataArray ()> Size: 8B
array(0.5)


**Example 4**: Weight and array

In [7]:
fcst_xr = xr.DataArray([3, 4, 5, 6, 7])
obs_xr = xr.DataArray([2, 3, 4, 5, 6])
weights = xr.DataArray(np.array([1, 2, 3, 2, 1]))
nse_weights = nse(fcst_xr, obs_xr, weights=weights)
print("NSE with weights types:", nse_weights)

NSE with weights types: <xarray.DataArray ()> Size: 8B
array(0.25)


**Example 5**: 2D Array: time and station

In [8]:
def create_synthetic_2d_data():
    # Define dimensions
    time = pd.date_range("2024-01-01", periods=5, freq="D")
    stations = ["Station1", "Station2", "Station3"]

    # Use specified forecast and observed values
    forecast_data = np.array(
        [[3, 4, 5, 6, 7], [3, 4, 5, 6, 7], [3, 4, 5, 6, 7]]
    ).T  # Transpose to align with dimensions (time, station)
    observed_data = np.array(
        [[2, 3, 4, 5, 6], [2, 3, 4, 5, 6], [2, 3, 4, 5, 6]]
    ).T  # Transpose to align with dimensions (time, station)

    # Create forecast DataArray
    forecast_da = xr.DataArray(
        forecast_data, coords={"time": time, "station": stations}, dims=["time", "station"], name="forecast"
    )

    # Create observed DataArray
    observed_da = xr.DataArray(
        observed_data, coords={"time": time, "station": stations}, dims=["time", "station"], name="observed"
    )

    return forecast_da, observed_da

In [20]:
# Create synthetic forecast and observed DataArrays
fcst_xr, obs_xr = create_synthetic_2d_data()

# Calculate the NSE for the test case, reducing over time - giving one value per station
nse_value = nse(fcst_xr, obs_xr, reduce_dims="time")
print("NSE for station:", nse_value)

NSE for station: <xarray.DataArray (station: 3)> Size: 24B
array([0.5, 0.5, 0.5])
Coordinates:
  * station  (station) <U8 96B 'Station1' 'Station2' 'Station3'


**Example 6**: 3D Array: Ensemble, Station and Time

In [10]:
def create_synthetic_3d_data():
    # Define dimensions
    time = pd.date_range("2024-01-01", periods=5, freq="D")
    stations = ["Station1", "Station2", "Station3"]
    ensemble = ["Ensemble1", "Ensemble2", "Ensemble3"]

    # Use specified forecast and observed values
    forecast_data = np.array(
        [[3, 4, 5, 6, 7], [3, 4, 5, 6, 7], [3, 4, 5, 6, 7]]
    ).T  # Transpose to align with dimensions (time, station)
    observed_data = np.array(
        [[2, 3, 4, 5, 6], [2, 3, 4, 5, 6], [2, 3, 4, 5, 6]]
    ).T  # Transpose to align with dimensions (time, station)

    # Repeat data for each ensemble member
    forecast_data = np.repeat(forecast_data[np.newaxis, ...], len(ensemble), axis=0)
    observed_data = np.repeat(observed_data[np.newaxis, ...], len(ensemble), axis=0)

    # Create forecast DataArray
    forecast_da = xr.DataArray(
        forecast_data,
        coords={"ensemble": ensemble, "time": time, "station": stations},
        dims=["ensemble", "time", "station"],
        name="forecast",
    )

    # Create observed DataArray
    observed_da = xr.DataArray(
        observed_data,
        coords={"ensemble": ensemble, "time": time, "station": stations},
        dims=["ensemble", "time", "station"],
        name="observed",
    )

    return forecast_da, observed_da

In [24]:
fcst_xr, obs_xr = create_synthetic_3d_data()

# Calculate the NSE for the test case, this time over the station, giving one value per time and ensemble
nse_value = nse(fcst_xr, obs_xr, reduce_dims="station")
print("NSE for station:", nse_value)

NSE for station: <xarray.DataArray (ensemble: 3, time: 5)> Size: 120B
array([[-inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf]])
Coordinates:
  * ensemble  (ensemble) <U9 108B 'Ensemble1' 'Ensemble2' 'Ensemble3'
  * time      (time) datetime64[ns] 40B 2024-01-01 2024-01-02 ... 2024-01-05


/home/nvr90/repos/scores_nr/src/scores/continuous/nse_impl.py:175: UserWarning: 
    Possible divide by zero: at least one element in the reduced obs variance array is 0. Any divide
    by zero entries will be filled in as `np.nan` if the forecast error is also 0, otherwise it will
    be `-np.inf`. This is so that any other valid entries are still computed and returned. The user
    should still verify that zero obs variance is expected for the given input data.
    
  warnings.warn(NseUtils.WARN_ZERO_OBS_VARIANCE)


**Warning: Divide by zero**

Note that we got a divide by zero warning. This is _expected_ since for each `(ensemble, time)` the value is the _same_. Recall that the NSE has the observation variance as the denominator. If the obs values are the same this implies that the variance is zero, we'd then have $\text{NSE} = 1 - \frac{\text{fcst\_error}}{0} = 1 - \infty = -\infty$. Usually the data isn't this contrived and we may still want to use the partial results - despite the ocassional divide by zero. To facilitate this, a warning is thrown instead of an error.

In [38]:
# We can briefly check the zero obs hypothesis
obs_xr.var(["station"])

<xarray.DataArray 'observed' (ensemble: 3, time: 5)> Size: 120B
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])
Coordinates:
  * ensemble  (ensemble) <U9 108B 'Ensemble1' 'Ensemble2' 'Ensemble3'
  * time      (time) datetime64[ns] 40B 2024-01-01 2024-01-02 ... 2024-01-05

**Example 7**: 3D Array: Time, Station and lead times

In [65]:
import numpy as np
import pandas as pd
import xarray as xr


def create_synthetic_deterministic_data():
    # Define dimensions
    time = pd.date_range("2024-01-01", "2024-01-31", freq="D")
    stations = ["Station1", "Station2", "Station3", "Station4", "Station5"]
    lead_times_forecast = np.arange(1, 8)  # Lead times from 1 to 7 for forecast
    lead_times_observed = np.array([1])  # Lead time of 1 day for observed data

    # Generate synthetic hydrograph data
    np.random.seed(0)  # For reproducibility

    # Base sine wave to simulate periodic streamflow variations
    days = len(time)
    base_flow = 150 + 50 * np.sin(2 * np.pi * np.arange(days) / days)  # Mean of 150, amplitude of 50

    # Reshape base_flow to match dimensions (31, 1, 1) for broadcasting
    base_flow = base_flow[:, np.newaxis, np.newaxis]

    # Adding random fluctuations around the base flow for forecast and observed data
    forecast_data = np.clip(base_flow + 20 * np.random.randn(days, len(stations), len(lead_times_forecast)), 0, 300)
    observed_data = np.clip(base_flow + 20 * np.random.randn(days, len(stations), len(lead_times_observed)), 0, 300)

    # Create forecast DataArray
    forecast_da = xr.DataArray(
        forecast_data,
        coords={"time": time, "station": stations, "lead_time": lead_times_forecast},
        dims=["time", "station", "lead_time"],
        name="forecast",
    )

    # Create observed DataArray
    observed_da = xr.DataArray(
        observed_data,
        coords={"time": time, "station": stations, "lead_time": lead_times_observed},
        dims=["time", "station", "lead_time"],
        name="observed",
    )

    return forecast_da, observed_da

In [69]:
# Display the DataArrays
# Create forecast and observed DataArrays
forecast_da, observed_da = create_synthetic_deterministic_data()
print(f"obs shape: {observed_da.shape}, fcst shape: {forecast_da.shape}")


obs shape: (31, 5, 1), fcst shape: (31, 5, 7)


(31, 5, 7)

**NSE Calculation along Lead time**

So far we've been only passing a string to `reduce_dims`, as mentioned in the introduction of this tutorial, the indexers can span multiple dimensions. Suppose we would like to find NSE for each lead time, we simply need to pass `reduce_dims=("station", "time")` alternatively, `preserve_dims="lead_time"` should also work. Before doing so, we need to broadcast the two arrays prior to computations. Normally, this will happen automatically - but fill it with NaNs when broadcasted.

In [88]:
print("NaN entries:")
display(sum(np.isnan(xr.broadcast(forecast_da, observed_da)[1])))
print("NaN entries with xarray broadcasting:")
display((forecast_da - observed_da).shape)
print("NaN entries with `.values` and numpy broadcasting:")
display(sum(np.isnan(np.broadcast_to(observed_da, forecast_da.shape))))
display((forecast_da.values - observed_da.values).shape)

NaN entries:


<xarray.DataArray 'observed' (station: 5, lead_time: 7)> Size: 280B
array([[ 0, 31, 31, 31, 31, 31, 31],
       [ 0, 31, 31, 31, 31, 31, 31],
       [ 0, 31, 31, 31, 31, 31, 31],
       [ 0, 31, 31, 31, 31, 31, 31],
       [ 0, 31, 31, 31, 31, 31, 31]])
Coordinates:
  * station    (station) <U8 160B 'Station1' 'Station2' ... 'Station5'
  * lead_time  (lead_time) int64 56B 1 2 3 4 5 6 7
    time       datetime64[ns] 8B 2024-01-31

NaN entries with xarray broadcasting:


(31, 5, 1)

NaN entries with `.values` and numpy broadcasting:


array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

(31, 5, 7)

**Proper broadcasting**

The reason this happens is because of the strict coordinate defintions. To circumvent this we actually need to use numpy's broadcast method, so that we are dealing with an attribute-less data-array. Since `nse` relies on `mse` for performing any computations under-the-hood, where the latter relies on using `xarray` for computational API calls rather than `numpy`. If we do want to mimic `numpy`'s method we just have to use `numpy` to call broadcast instead.

> **Caution**
>
> Using numpy for broadcasting will almost certainly load everything to memory. However, unlike `xarray` which relies on merge strategies, numpy basically repeats data if the dimensions are compatible.
>
> In any case - broadcasting is not always trivial - especially for large datasets, and `nse` (and by extension `mse`) in particular do not guarentee broadcasted operations, instead they trim the operations to coordinates that are compatible

In [122]:
observed_np_broadcast = np.broadcast_to(observed_da, forecast_da.shape)
observed_da_broadcast = observed_da.broadcast_like(forecast_da)
observed_da_broadcast = observed_da_broadcast.copy(data=observed_np_broadcast)

# to breakdown the above:
# - np.broadcast_to: takes the observed_da and broadcasts it to the forecast_da's shape - it repeats the data if compatible.
# - DataArray.broadcast_like: is a relatively cheap operation that just setsup extra space to match the forecast data array's shape
# - .copy: creates a shallow copy, but it's mostly used for its "data" argument which can be used to update the entries in the newly broadcasted array.
# Now if we look at the shape:
print("broadcasted obs shape:")
display(observed_da_broadcast.shape)
print("Check index 0 against index 1 have the same repeated values")
display(np.all(observed_da_broadcast[:,:,0] == observed_da_broadcast[:,:,1]).values)
print("Check index 0 against last index (-1) have the same repeated values")
display(np.all(observed_da_broadcast[:,:,0] == observed_da_broadcast[:,:,-1]).values)
print("Now we have repeating obs... let's compute NSE\n")

print("\n------------------------------")
print(" NSE broadcasted using xarray")
print("------------------------------\n")

# note: see how we can specify reduce_dims on an array of dimensions instead of a single string
display(nse(forecast_da, observed_da, reduce_dims=["time", "station"]))
print("Note that this still works, but we only get a single output - for lead time 1...")

print("\n-----------------------------")
print(" NSE broadcasted using numpy")
print("-----------------------------\n")
display(nse(forecast_da, observed_da_broadcast, reduce_dims=["time", "station"]))
print("...However using the numpy based broadcasting we get all 7 leadtimes")
print("Note we could have also used `preserve_dims='lead_time'` to get the same result:")
display(nse(forecast_da, observed_da_broadcast, preserve_dims="lead_time"))

broadcasted obs shape:


(31, 5, 7)

Check index 0 against index 1 have the same repeated values


array(True)

Check index 0 against last index (-1) have the same repeated values


...

<xarray.DataArray (lead_time: 1)> Size: 8B
array([-0.32824831])
Coordinates:
  * lead_time  (lead_time) int64 8B 1

Note that this still works, but we only get a single output - for lead time 1...

-----------------------------
 NSE broadcasted using numpy
-----------------------------



<xarray.DataArray (lead_time: 7)> Size: 56B
array([-0.32824831, -0.38219119, -0.26666195, -0.37203328, -0.27325163,
       -0.30057656, -0.33785027])
Coordinates:
  * lead_time  (lead_time) int64 56B 1 2 3 4 5 6 7

...However using the numpy based broadcasting we get all 7 leadtimes
Note we could have also used `preserve_dims='lead_time'` to get the same result:


<xarray.DataArray (lead_time: 7)> Size: 56B
array([-0.32824831, -0.38219119, -0.26666195, -0.37203328, -0.27325163,
       -0.30057656, -0.33785027])
Coordinates:
  * lead_time  (lead_time) int64 56B 1 2 3 4 5 6 7

**Example 8**: 4D Array: Time, Station, lead times and ensemble

In [123]:
def create_synthetic_ensemble_data():
    # Define dimensions
    time = pd.date_range("2024-01-01", "2024-01-31", freq="D")
    stations = ["Station1", "Station2", "Station3", "Station4", "Station5"]
    lead_times_forecast = np.arange(1, 8)  # Lead times from 1 to 7 for forecast
    lead_times_observed = np.array([1])  # Lead time of 1 day for observed data
    ensemble = [f"Ensemble{i}" for i in range(1, 21)]  # Ensemble members from Ensemble1 to Ensemble20

    # Generate synthetic hydrograph data
    np.random.seed(0)  # For reproducibility

    # Create base flow as a constant value of 30 cumec
    base_flow = 30

    # Adding random fluctuations around the base flow for forecast and observed data
    forecast_data = base_flow + np.random.randint(
        0, 301, size=(len(time), len(stations), len(lead_times_forecast), len(ensemble))
    )
    observed_data = base_flow + np.random.randint(0, 301, size=(len(time), len(stations), len(lead_times_observed)))

    # Clip the data to ensure it stays within a certain range
    forecast_data = np.clip(forecast_data, 0, 300)
    observed_data = np.clip(observed_data, 0, 300)

    # Create forecast DataArray
    forecast_da = xr.DataArray(
        forecast_data,
        coords={"time": time, "station": stations, "lead_time": lead_times_forecast, "ensemble": ensemble},
        dims=["time", "station", "lead_time", "ensemble"],
        name="forecast",
    )

    # Create observed DataArray
    observed_da = xr.DataArray(
        observed_data,
        coords={"time": time, "station": stations, "lead_time": lead_times_observed},
        dims=["time", "station", "lead_time"],
        name="observed",
    )

    return forecast_da, observed_da

In [119]:
# Create forecast and observed DataArrays
forecast_ensemble_da, observed_da = create_synthetic_ensemble_data()

# Display the DataArrays
# print(forecast_ensemble_da)

In [120]:
print(forecast_ensemble_da.shape)
print(observed_da.shape)

(31, 5, 7, 20)
(31, 5, 1)


<xarray.DataArray 'forecast' (time: 31, station: 5, lead_time: 7, ensemble: 20)>

<xarray.DataArray 'observed' (time: 31, station: 5, lead_time: 1)>

**Dask**

Sometimes with large dimensional arrays, it _may not_ be wise to use numpy to manually broadcast operations. While it maybe fast - it uses up a lot of memory.

Here's where dask can help instead.

In [18]:
# Initialize an empty list to store results
results_list = []

# Iterate through each ensemble member and lead time
for imember in forecast_ensemble_da["ensemble"].values:
    for ilead in forecast_ensemble_da["lead_time"].values:
        # Select the forecast data for the current ensemble member and lead time
        cur_fcst_da = forecast_ensemble_da.sel(ensemble=imember, lead_time=ilead)

        # Calculate NSE value for the selected forecast data and observed data
        nse_value = nse(cur_fcst_da, observed_da).values

        # Append the results as a dictionary to the list
        results_list.append({"ensemble": imember, "lead_time": ilead, "NSE": nse_value})

# Create a DataFrame from the list of dictionaries
nse_results = pd.DataFrame(results_list)

In [19]:
# Print the DataFrame
print(nse_results.head(3))

    ensemble  lead_time                  NSE
0  Ensemble1          1  -1.1721371704833192
1  Ensemble1          2  -1.0448237401444582
2  Ensemble1          3   -1.060897482889457
